In [152]:
import datetime as dt
from statistics import mean
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm import tqdm
import portion as P
length_packet = 362

def rtt_plot(df):
    l1 = []
    l2 = []
    for i in tqdm(range(len(df))):
        if not np.isnan(df.loc[i, "tcp.analysis.ack_rtt"]):
            l1.append(df.loc[i, "frame.time"])
            l2.append(df.loc[i, "tcp.analysis.ack_rtt"])
    print("mean:", np.mean(l2))
    plt.plot(l1, l2)
    plt.show()



def get_seq_time():
    rxdf = pd.read_csv(rx_pcap, sep='@')
    rxdf.loc[:, "frame.time"] = pd.to_datetime(rxdf.loc[:, r"frame.time"]).dt.tz_localize(None)
    # map seq to time
    
    rseq_time_list  = []
    tseq_time_list  = []

    for i in tqdm(range(len(rxdf))):
        if rxdf.loc[i, "tcp.len"] >= length_packet and rxdf.loc[i, "ip.src"] == "140.112.20.183":
            bias = rxdf.loc[i, "tcp.payload"].find("0000000062")
            dup_num = (rxdf.loc[i, "tcp.len"] -bias) // length_packet
            packet_time = rxdf.loc[i, "frame.time"]
            for j in range(dup_num):
                seq = int(rxdf.loc[i, "tcp.payload"][bias+32+j*length_packet*2:bias+48+j*length_packet*2], 16)
#                 print(seq)
                rseq_time_list.append((seq, packet_time))

def get_client_ul_seq(txdf):
    length_packet = 362
    tseq_time_dict = {}
    ip_set = set(txdf.loc[:, "ip.src"])
    ip_set.remove("140.112.20.183")
    for i in tqdm(range(len(txdf))):
        if txdf.loc[i, "tcp.len"] >= length_packet and txdf.loc[i, "ip.dst"] == "140.112.20.183":
            bias = txdf.loc[i, "tcp.payload"].find("0000018")
            if bias == -1:
                continue
            dup_num = (txdf.loc[i, "tcp.len"] -bias) // length_packet
            packet_time = txdf.loc[i, "frame.time"]
            for j in range(dup_num):
                seq = int(txdf.loc[i, "tcp.payload"][bias+16+j*length_packet*2:bias+24+j*length_packet*2], 16)
                born_time = int(txdf.loc[i, "tcp.payload"][bias+j*length_packet*2:bias+16+j*length_packet*2], 16)/1000
                born_time = dt.datetime.fromtimestamp(born_time)
                delay = (packet_time - born_time).total_seconds()
                if delay > 0:
                    key = (txdf.loc[i, "ip.src"] , txdf.loc[i, "tcp.srcport"],)
                    if key not in tseq_time_dict:
                        tseq_time_dict[key] = []
                    tseq_time_dict[key].append((seq, packet_time, delay, txdf.loc[i, "frame.number"]))
    for key in tseq_time_dict:
        tseq_time_dict[key].sort(key = lambda x:x[0])
        
    return tseq_time_dict

def get_server_ul_seq(rxdf):
    length_packet = 362
    rseq_time_dict = {}
    for i in tqdm(range(len(rxdf))):
        if rxdf.loc[i, "tcp.len"] >= length_packet:
            bias = rxdf.loc[i, "tcp.payload"].find("0000018")
            if bias == -1:
                continue
            dup_num = (rxdf.loc[i, "tcp.len"] -bias) // length_packet
            packet_time = rxdf.loc[i, "frame.time"]
            for j in range(dup_num):
                seq = int(rxdf.loc[i, "tcp.payload"][bias+16+j*length_packet*2:bias+24+j*length_packet*2], 16)
                born_time = int(rxdf.loc[i, "tcp.payload"][bias+j*length_packet*2:bias+16+j*length_packet*2], 16)/1000
                born_time = dt.datetime.fromtimestamp(born_time)
                delay = (packet_time - born_time).total_seconds()
                if delay > -10:
                    key = (rxdf.loc[i, "ip.src"] , rxdf.loc[i, "tcp.srcport"],)
                    if key not in rseq_time_dict:
                        rseq_time_dict[key] = []
                    rseq_time_dict[key].append((seq, packet_time, delay, rxdf.loc[i, "frame.number"]))
    for key in rseq_time_dict:
        rseq_time_dict[key].sort(key = lambda x:x[0])
        
    return rseq_time_dict

def get_ret(pcapdf):
    # get retransmision time list
    time_list = []
    for i in range(len(pcapdf)):
        if "Retran" in pcapdf.loc[i, "_ws.col.Info"]:
            time_list.append(pcapdf.loc[i, "frame.time"])
    time_list = pd.to_datetime(time_list)
    return time_list

def get_ul_rtt(pcapdf):
    l1 = []
    for i in tqdm(range(len(pcapdf))):
        if not np.isnan(pcapdf.loc[i, "tcp.analysis.ack_rtt"]) and pcapdf.loc[i, "ip.dst"] == "140.112.20.183":
            l1.append((pcapdf.loc[i, "frame.time"], pcapdf.loc[i, "tcp.analysis.ack_rtt"]))
    return np.array(l1)

def get_dl_rtt(pcapdf):
    l1 = []
    for i in tqdm(range(len(pcapdf))):
        if not np.isnan(pcapdf.loc[i, "tcp.analysis.ack_rtt"]):
            l1.append((pcapdf.loc[i, "frame.time"], pcapdf.loc[i, "tcp.analysis.ack_rtt"]))
    return np.array(l1)

def latency_impulse(latency_list, t):
    return sum(latency_list[:, 0] > t) / len(latency_list)

def read_pcapcsv(filename):
    df = pd.read_csv(filename, sep='@')
    df.loc[:, "frame.time"] = pd.to_datetime(df.loc[:, r"frame.time"]).dt.tz_localize(None)
    return df

def UL_latency(rseq_time_list, tseq_time_list):
    latency_list = []
    i = 0
    j = 0
    
    while i < len(rseq_time_list) and j < len(tseq_time_list):
        
        if rseq_time_list[i][0] * 100 < tseq_time_list[j][0] or rseq_time_list[i][0] > tseq_time_list[j][0] * 100:
            break

        if rseq_time_list[i][0] < tseq_time_list[j][0]:
            i += 1
        elif rseq_time_list[i][0] > tseq_time_list[j][0]:
            j += 1
        else:
#             print(rseq_time_list[i], tseq_time_list[j])
            latency_list.append(((rseq_time_list[i][1] - tseq_time_list[j][1]).total_seconds(), tseq_time_list[i][1], rseq_time_list[j][0]))
            i += 1
            j += 1
    latency_list = np.array(latency_list)
    latency_list[:, 0] -= latency_list[:, 0].min()
#     latency_list[:, 0] += 0.01
    plt.title("latency")
    plt.ylabel("seconds")
    plt.plot(latency_list[:, 1], latency_list[:, 0])
    plt.show()
    print(len(latency_list))
    print(np.mean(latency_list[:, 0]))
    return latency_list

def mp_latency(l1, l2):
    # (latency, time, seq)
    
    i = 0
    j = 0
    latency_list = []

    
    while i < len(l1) and j < len(l2):
        
        if l1[i][2] * 100 < l2[j][2] or l1[i][2] > l2[j][2] * 100:
            break

        if l1[i][2] < l2[j][2]:
            i += 1
        elif l1[i][2] > l2[j][2]:
            j += 1
        else:
            print(l1[i], l2[j])
            # latency, time, seq
#             print(l1[i][0], l2[j][0], np.min())
            latency_list.append((np.min((l1[i][0], l2[j][0])), l2[j][1], l2[j][2]))
            i += 1
            j += 1
    latency_list = np.array(latency_list)
#     latency_list[:, 0] -= latency_list[:, 0].min()
#     latency_list[:, 0] += 0.01
    plt.title("latency")
    plt.ylabel("seconds")
    plt.plot(latency_list[:, 0], latency_list[:, 2])
    plt.show()
    print(len(latency_list))
    print(np.mean(latency_list[:, 0]))
    return latency_list



In [153]:
def mi_event_parsing(miinfofile, nr_time_intervals):

    nr_pci = None
    
    lte_4G_handover_list = []   #4G 狀態下LTE eNB 的 handover
    
    nr_setup_list = []          #gNB cell addition
    nr_handover_list = []       #gNB cell changes (eNB stays the same)
    nr_removal_list = []        #gNB cell removal
        
    lte_5G_handover_list = []   #(eNB1, gNB1) -> (eNB2, gNB1) #gNB stays the same
    nr_lte_handover_list = []   #both NR cell and LTE cell have handover
    
    scg_failure_list = []       #gNB handover failure
    reestablish_list = []       #eNB handover failure
    
    nr_handover = 0
    nr_handover_start_index = None
    lte_handover = 0
    lte_handover_start_index = None
    nr_release = 0
    nr_release_start_index = None
    
    handover_num = 0
    
    #initialization
    #----------------------------------------------------------------
    if miinfofile.loc[0, "time"] in nr_time_intervals:
        nr_pci = 1000000            #先設置一個數 dummy nr pci
    
    for i in range(len(miinfofile)):       
        if miinfofile.loc[i, "nr-rrc.t304"]:
            if nr_handover == 0:    
                nr_handover = 1
                nr_handover_start_index = i
                
        if miinfofile.loc[i, "lte-rrc.t304"]:
            if lte_handover == 0:
                lte_handover = 1
                lte_handover_start_index = i
                
        if miinfofile.loc[i, "nr-Config-r15: release (0)"]:
            if nr_release == 0:
                nr_release = 1
                nr_release_start_index = i
           
        if (nr_handover or lte_handover or nr_release) and miinfofile.loc[i, "rrcConnectionReconfigurationComplete"]:
            handover_num +=1
        
        
        #handover 種類分類
        #------------------------------------------------------------------------------
        if lte_handover and not nr_handover and miinfofile.loc[i, "rrcConnectionReconfigurationComplete"]:  # just lte cell handover event
            lte_handover = 0
            lte_4G_handover_list.append([miinfofile.loc[lte_handover_start_index, "time"], miinfofile.loc[i, "time"]])
            
        if nr_handover and not lte_handover and miinfofile.loc[i, "rrcConnectionReconfigurationComplete"]:  # just nr cell handover event
            nr_handover = 0
            if miinfofile.loc[nr_handover_start_index, "dualConnectivityPHR: setup (1)"]:     #This if-else statement classifies whether it is nr addition or nr handover
                nr_setup_list.append([miinfofile.loc[nr_handover_start_index, "time"], miinfofile.loc[i, "time"]])       
            else:
                nr_handover_list.append([miinfofile.loc[nr_handover_start_index, "time"], miinfofile.loc[i, "time"]])
                
            #additional judgement:
            #----------------------------
            #if miinfofile.loc[nr_handover_start_index, "dualConnectivityPHR: setup (1)"] and nr_pci != None:
            #    print("Warning: dualConnectivityPHR setup may not mean nr cell addition", mi_file, i)
            #if miinfofile.loc[nr_handover_start_index, "dualConnectivityPHR: setup (1)"]==0 and not (nr_pci != None and nr_pci != miinfofile.loc[nr_handover_start_index, "nr_pci"]): 
            #    print("Warning: nr-rrc.t304 without dualConnectivityPHR setup may not mean nr cell handover", mi_file, i, nr_handover_start_index, miinfofile.loc[nr_handover_start_index, "nr_pci"], nr_pci)
                
            #nr_pci update lte_handover_start_time
            nr_pci = miinfofile.loc[nr_handover_start_index, "nr_pci"]
            
            
        if lte_handover and nr_handover and miinfofile.loc[i, "rrcConnectionReconfigurationComplete"]:      # both nr cell and lte cell handover event
            lte_handover = 0
            nr_handover = 0
            if nr_pci == miinfofile.loc[lte_handover_start_index, "nr_pci"]:
                lte_5G_handover_list.append([miinfofile.loc[lte_handover_start_index, "time"], miinfofile.loc[i, "time"]])
            else:
                nr_lte_handover_list.append([miinfofile.loc[lte_handover_start_index, "time"], miinfofile.loc[i, "time"]])
            
            #nr_pci update
            nr_pci = miinfofile.loc[lte_handover_start_index, "nr_pci"]
            
        if nr_release and miinfofile.loc[i, "rrcConnectionReconfigurationComplete"]:
            nr_pci = None
            nr_release=0
            nr_removal_list.append([miinfofile.loc[nr_release_start_index, "time"], miinfofile.loc[i, "time"]])
        if miinfofile.loc[i, "scgFailureInformationNR-r15"]:
            scg_failure_list.append([miinfofile.loc[i, "time"], miinfofile.loc[i, "time"]])
        if miinfofile.loc[i, "rrcConnectionReestablishmentRequest"]:
            reestablish_list.append([miinfofile.loc[i, "time"], miinfofile.loc[i, "time"]])
            
    return [lte_4G_handover_list, nr_setup_list, nr_handover_list, nr_removal_list, lte_5G_handover_list, nr_lte_handover_list, scg_failure_list, reestablish_list], handover_num

#This function firstly generates three intervals for each [handover_start, handover_end] in handover_list
#   a. before the handover_start event: [handover_start-second, handover_start]
#   b. during the handover events: [handover_start, handover_end]
#   c. after the handover_end event: [handover_end, handover_end+second]
#Then, it returns the overall intervals before/during/after the handover event
#--------------------------------------------------------------------
def get_before_during_after_intervals(handover_list, second):
    before_handover_intervals = P.empty()
    handover_intervals = P.empty()
    after_handover_intervals = P.empty()
    for i in range(len(handover_list)):  
        interval = P.closed( handover_list[i][0]-dt.timedelta(seconds=second), handover_list[i][0] )
        before_handover_intervals = before_handover_intervals | interval
        
        interval = P.closed( handover_list[i][0], handover_list[i][1] )
        handover_intervals = handover_intervals | interval
        
        interval = P.closed( handover_list[i][1], handover_list[i][1]+dt.timedelta(seconds=second) )
        after_handover_intervals = after_handover_intervals | interval    
    return before_handover_intervals, handover_intervals, after_handover_intervals
   
#This function returns the overall length of the intervals
#--------------------------------------------------------------------
def get_sum_intervals(intervals):
    if intervals.empty:
        return 0
    sum = 0
    for x in intervals:
        sum += (x.upper - x.lower)/dt.timedelta(seconds=1)
    return sum

event_names = [
    "lte_4G_handover",
    "nr_setup",
    "nr_handover",
    "nr_removal",
    "lte_5G_handover",
    "nr_lte_handover_list",
    "scg_failure",
    "reestablish"
]

column_names = []

for i in range(len(event_names)-2):
    column_names += ["before_"+event_names[i]+"_intervals", "during_"+event_names[i]+"_intervals", "after_"+event_names[i]+"_intervals"]
for i in range(len(event_names)-2, len(event_names)):
    column_names += ["before_"+event_names[i]+"_intervals", "after_"+event_names[i]+"_intervals"]
    
column_names += ["nr_time_intervals", "weak_nr_intervals", "weak_lte_intervals"]
print("column_names=", column_names)

column_names_time = ["time_"+ c for c in column_names]
print(column_names_time)



column_names= ['before_lte_4G_handover_intervals', 'during_lte_4G_handover_intervals', 'after_lte_4G_handover_intervals', 'before_nr_setup_intervals', 'during_nr_setup_intervals', 'after_nr_setup_intervals', 'before_nr_handover_intervals', 'during_nr_handover_intervals', 'after_nr_handover_intervals', 'before_nr_removal_intervals', 'during_nr_removal_intervals', 'after_nr_removal_intervals', 'before_lte_5G_handover_intervals', 'during_lte_5G_handover_intervals', 'after_lte_5G_handover_intervals', 'before_nr_lte_handover_list_intervals', 'during_nr_lte_handover_list_intervals', 'after_nr_lte_handover_list_intervals', 'before_scg_failure_intervals', 'after_scg_failure_intervals', 'before_reestablish_intervals', 'after_reestablish_intervals', 'nr_time_intervals', 'weak_nr_intervals', 'weak_lte_intervals']
['time_before_lte_4G_handover_intervals', 'time_during_lte_4G_handover_intervals', 'time_after_lte_4G_handover_intervals', 'time_before_nr_setup_intervals', 'time_during_nr_setup_interva

In [154]:
def analysis_target_dir_dl(dirpath , outputcsvfilename):
    outputfilecsv = open(outputcsvfilename, mode='w')
    cellinfodir = sorted(os.listdir(os.path.join(dirpath, "cellinfo")))
    mi2logdir = sorted(os.listdir(os.path.join(dirpath, "mi2log")))
    # pcapdir = sorted(os.listdir(os.path.join(dirpath, "pcap")))
    pcapdir = sorted(os.listdir(os.path.join(dirpath, "server_dl_pcap")))
    assert(len(cellinfodir) == len(mi2logdir) == len(pcapdir) )
    for cellinfo, mi2log, pcapcsv in zip(cellinfodir, mi2logdir, pcapdir):
        stable_drop = 0
        print(cellinfo, mi2log, pcapcsv)
        sum_intervals = [0] * len(column_names)
        sum_packet_loss = [0] * len(column_names)
        sum_stable_intervals = 0
        cellinfofile = pd.read_csv(os.path.join(os.path.join(dirpath, "cellinfo"), cellinfo))
        cellinfofile.loc[:, "Date"] = pd.to_datetime(cellinfofile.loc[:, "Date"])

        miinfofile = pd.read_csv(os.path.join(os.path.join(dirpath, "mi2log"), mi2log))
        miinfofile.loc[:, "time"] = pd.to_datetime(miinfofile.loc[:, "time"]) + dt.timedelta(hours=8)

        # pcapdf = read_pcapcsv(os.path.join(os.path.join(dirpath, "pcap"), pcapcsv))
        pcapdf = read_pcapcsv(os.path.join(os.path.join(dirpath, "server_dl_pcap"), pcapcsv))
        #======================lost time (and latency)=============================
        # loss_time, latency = get_loss_latency(pcap)
        ret_list = get_ret(pcapdf)

        if len(ret_list) == 0:
            print("no ret")
            continue

        #======================before mi event parse, parse NR intervals first=====
        nr_time_intervals = P.empty()
        if cellinfofile.loc[0, "NR_SSRSRP"] != "-":
            nr_time_intervals = P.singleton(cellinfofile.loc[0, "Date"])
        for i in range(1, len(cellinfofile)):
            if cellinfofile.loc[i, "NR_SSRSRP"] != "-":
                nr_time_intervals = nr_time_intervals | P.openclosed(cellinfofile.loc[i-1, "Date"] , cellinfofile.loc[i, "Date"])
        weak_nr_intervals = P.empty()
        if cellinfofile.loc[0, "NR_SSRSRP"] != "-" and int(cellinfofile.loc[0, "NR_SSRSRP"]) <= -90:
            weak_nr_intervals = P.singleton(cellinfofile.loc[0, "Date"])
        for i in range(1, len(cellinfofile)):
            if cellinfofile.loc[i, "NR_SSRSRP"] != "-" and int(cellinfofile.loc[i, "NR_SSRSRP"]) <= -90:
                weak_nr_intervals = weak_nr_intervals | P.openclosed(cellinfofile.loc[i-1, "Date"] , cellinfofile.loc[i, "Date"])
        weak_lte_intervals = P.empty()
        if cellinfofile.loc[0, "LTE_RSRP"] != "-" and int(cellinfofile.loc[0, "LTE_RSRP"]) <= -90:
            weak_lte_intervals = P.singleton(cellinfofile.loc[0, "Date"])
        for i in range(1, len(cellinfofile)):
            if cellinfofile.loc[i, "LTE_RSRP"] != "-" and int(cellinfofile.loc[i, "LTE_RSRP"]) <= -90:
                weak_lte_intervals = weak_lte_intervals | P.openclosed(cellinfofile.loc[i-1, "Date"] , cellinfofile.loc[i, "Date"])

        #======================mi event parse======================================  
        handover_event_lists, handover_num = mi_event_parsing(miinfofile, nr_time_intervals)

        #======================making intervals====================================
        intervals = []

        for handover_event_list in handover_event_lists[:-2]:
            before_intervals, during_intervals, after_intervals = get_before_during_after_intervals(handover_event_list, 1)     #抓取前後一秒
            intervals += [before_intervals, during_intervals, after_intervals].copy()
        for handover_event_list in handover_event_lists[-2:]:
            before_intervals, _, after_intervals = get_before_during_after_intervals(handover_event_list, 3)                    #抓取前後三秒
            intervals += [before_intervals, after_intervals].copy()    

        intervals += [nr_time_intervals, weak_nr_intervals, weak_lte_intervals]
        #================================judgement=================================
        out = pd.DataFrame(columns = column_names, dtype=object)

        #------check time-------
        start_time = miinfofile.loc[0, "time"]
        if cellinfofile.loc[0, "Date"] > start_time:
            start_time = cellinfofile.loc[0, "Date"]
        if ret_list[0] > start_time:
            start_time = ret_list[0]
        end_time = miinfofile.loc[len(miinfofile)-1, "time"]
        if cellinfofile.loc[len(cellinfofile)-1, "Date"] < end_time:
            end_time = cellinfofile.loc[len(cellinfofile)-1, "Date"]
        if ret_list[-1] < end_time:
            end_time = ret_list[-1]
        ###
        for loss_index in range(len(ret_list)):    
            stable = True
            if ret_list[loss_index] < start_time:
                continue
            if ret_list[loss_index] > end_time:
                continue
            
            types = [0]*len(intervals)
            for i in range(len(intervals)):
                if ret_list[loss_index] in intervals[i]:
                    types[i] = 1
                    stable = False
            if stable:
                stable_drop += 1
            out.loc[loss_index] = types  
        #output: number of packet loss, number of packet loss under each type, handover num, overall experiment time, file names 
        output = [len(out)] + [len(out.loc[ out[column_names[i]]==1 ]) for i in range(len(column_names)-3)] + [handover_num, (end_time-start_time)/dt.timedelta(seconds=1)]

        for i in range(len(column_names)):
            intervals[i] = intervals[i] & P.closed(start_time, end_time)
            sum_intervals[i] += get_sum_intervals(intervals[i])

        stable_intervals = P.closed(start_time, end_time)
        for i in range(len(column_names)-3):
            stable_intervals = stable_intervals - intervals[i]
        sum_stable_intervals += get_sum_intervals(stable_intervals)

        for i in range(len(column_names)-3):
            sum_packet_loss[i] += len(out.loc[ out[column_names[i]]==1 ])

        #User can decide whether save the packet loss classification as csv file
        out.to_csv("all_loss_classify_7.csv", mode='a')
        sum_intervals_list = [sum_intervals[i] for i in range(len(column_names)-3)]
        # print(sum_stable_intervals)
        # print([sum_intervals[i] for i in range(len(column_names)-3)])
        outlist = []
        # num of packetloss , num of packetloss under each type, rate(num / time) , #HO, # experiment duration, 3files
        outlist = [len(out), stable_drop] + sum_packet_loss[:-3] + [sum_stable_intervals] + sum_intervals_list + [handover_num, (end_time-start_time)/dt.timedelta(seconds=1)] +\
            [cellinfo, mi2log, pcapcsv]
        outlist = [str(x) for x in outlist]
        outputfilecsv.write(",".join(outlist)+'\n')
        print("-"*30)
    outputfilecsv.close()

In [155]:
def analysis_target_dir_ul(dirpath , outputcsvfilename):
    outputfilecsv = open(outputcsvfilename, mode='w')
    cellinfodir = sorted(os.listdir(os.path.join(dirpath, "cellinfo")))
    mi2logdir = sorted(os.listdir(os.path.join(dirpath, "mi2log")))
    pcapdir = sorted(os.listdir(os.path.join(dirpath, "pcap")))
    # pcapdir = sorted(os.listdir(os.path.join(dirpath, "server_dl_pcap")))
    assert(len(cellinfodir) == len(mi2logdir) == len(pcapdir) )
    for cellinfo, mi2log, pcapcsv in zip(cellinfodir, mi2logdir, pcapdir):
        stable_drop = 0
        print(cellinfo, mi2log, pcapcsv)
        sum_intervals = [0] * len(column_names)
        sum_packet_loss = [0] * len(column_names)
        sum_stable_intervals = 0
        cellinfofile = pd.read_csv(os.path.join(os.path.join(dirpath, "cellinfo"), cellinfo))
        cellinfofile.loc[:, "Date"] = pd.to_datetime(cellinfofile.loc[:, "Date"])

        miinfofile = pd.read_csv(os.path.join(os.path.join(dirpath, "mi2log"), mi2log))
        miinfofile.loc[:, "time"] = pd.to_datetime(miinfofile.loc[:, "time"]) + dt.timedelta(hours=8)

        pcapdf = read_pcapcsv(os.path.join(os.path.join(dirpath, "pcap"), pcapcsv))
        # pcapdf = read_pcapcsv(os.path.join(os.path.join(dirpath, "server_dl_pcap"), pcapcsv))
        #======================lost time (and latency)=============================
        # loss_time, latency = get_loss_latency(pcap)
        ret_list = get_ret(pcapdf)

        if len(ret_list) == 0:
            print("no ret")
            continue

        #======================before mi event parse, parse NR intervals first=====
        nr_time_intervals = P.empty()
        if cellinfofile.loc[0, "NR_SSRSRP"] != "-":
            nr_time_intervals = P.singleton(cellinfofile.loc[0, "Date"])
        for i in range(1, len(cellinfofile)):
            if cellinfofile.loc[i, "NR_SSRSRP"] != "-":
                nr_time_intervals = nr_time_intervals | P.openclosed(cellinfofile.loc[i-1, "Date"] , cellinfofile.loc[i, "Date"])
        weak_nr_intervals = P.empty()
        if cellinfofile.loc[0, "NR_SSRSRP"] != "-" and int(cellinfofile.loc[0, "NR_SSRSRP"]) <= -90:
            weak_nr_intervals = P.singleton(cellinfofile.loc[0, "Date"])
        for i in range(1, len(cellinfofile)):
            if cellinfofile.loc[i, "NR_SSRSRP"] != "-" and int(cellinfofile.loc[i, "NR_SSRSRP"]) <= -90:
                weak_nr_intervals = weak_nr_intervals | P.openclosed(cellinfofile.loc[i-1, "Date"] , cellinfofile.loc[i, "Date"])
        weak_lte_intervals = P.empty()
        if cellinfofile.loc[0, "LTE_RSRP"] != "-" and int(cellinfofile.loc[0, "LTE_RSRP"]) <= -90:
            weak_lte_intervals = P.singleton(cellinfofile.loc[0, "Date"])
        for i in range(1, len(cellinfofile)):
            if cellinfofile.loc[i, "LTE_RSRP"] != "-" and int(cellinfofile.loc[i, "LTE_RSRP"]) <= -90:
                weak_lte_intervals = weak_lte_intervals | P.openclosed(cellinfofile.loc[i-1, "Date"] , cellinfofile.loc[i, "Date"])

        #======================mi event parse======================================  
        handover_event_lists, handover_num = mi_event_parsing(miinfofile, nr_time_intervals)

        #======================making intervals====================================
        intervals = []

        for handover_event_list in handover_event_lists[:-2]:
            before_intervals, during_intervals, after_intervals = get_before_during_after_intervals(handover_event_list, 1)     #抓取前後一秒
            intervals += [before_intervals, during_intervals, after_intervals].copy()
        for handover_event_list in handover_event_lists[-2:]:
            before_intervals, _, after_intervals = get_before_during_after_intervals(handover_event_list, 3)                    #抓取前後三秒
            intervals += [before_intervals, after_intervals].copy()    

        intervals += [nr_time_intervals, weak_nr_intervals, weak_lte_intervals]
        #================================judgement=================================
        out = pd.DataFrame(columns = column_names, dtype=object)

        #------check time-------
        start_time = miinfofile.loc[0, "time"]
        if cellinfofile.loc[0, "Date"] > start_time:
            start_time = cellinfofile.loc[0, "Date"]
        if ret_list[0] > start_time:
            start_time = ret_list[0]
        end_time = miinfofile.loc[len(miinfofile)-1, "time"]
        if cellinfofile.loc[len(cellinfofile)-1, "Date"] < end_time:
            end_time = cellinfofile.loc[len(cellinfofile)-1, "Date"]
        if ret_list[-1] < end_time:
            end_time = ret_list[-1]
        ###
        for loss_index in range(len(ret_list)):    
            stable = True
            if ret_list[loss_index] < start_time:
                continue
            if ret_list[loss_index] > end_time:
                continue
            
            types = [0]*len(intervals)
            for i in range(len(intervals)):
                if ret_list[loss_index] in intervals[i]:
                    types[i] = 1
                    stable = False
            if stable:
                stable_drop += 1
            out.loc[loss_index] = types  
        #output: number of packet loss, number of packet loss under each type, handover num, overall experiment time, file names 
        output = [len(out)] + [len(out.loc[ out[column_names[i]]==1 ]) for i in range(len(column_names)-3)] + [handover_num, (end_time-start_time)/dt.timedelta(seconds=1)]

        for i in range(len(column_names)):
            intervals[i] = intervals[i] & P.closed(start_time, end_time)
            sum_intervals[i] += get_sum_intervals(intervals[i])

        stable_intervals = P.closed(start_time, end_time)
        for i in range(len(column_names)-3):
            stable_intervals = stable_intervals - intervals[i]
        sum_stable_intervals += get_sum_intervals(stable_intervals)

        for i in range(len(column_names)-3):
            sum_packet_loss[i] += len(out.loc[ out[column_names[i]]==1 ])

        #User can decide whether save the packet loss classification as csv file
        out.to_csv("all_loss_classify_7.csv", mode='a')
        sum_intervals_list = [sum_intervals[i] for i in range(len(column_names)-3)]
        # print(sum_stable_intervals)
        # print([sum_intervals[i] for i in range(len(column_names)-3)])
        outlist = []
        # num of packetloss , num of packetloss under each type, rate(num / time) , #HO, # experiment duration, 3files
        outlist = [len(out), stable_drop] + sum_packet_loss[:-3] + [sum_stable_intervals] + sum_intervals_list + [handover_num, (end_time-start_time)/dt.timedelta(seconds=1)] +\
            [cellinfo, mi2log, pcapcsv]
        outlist = [str(x) for x in outlist]
        outputfilecsv.write(",".join(outlist)+'\n')
        print("-"*30)
    outputfilecsv.close()

In [76]:
analysis_target_dir_dl('/home/wmnlab/D/xm04/0421' , 'xm04result.csv')

042122080537_new.csv diag_log_20220421_201721_b0e4a15ac89caff782b5f2af1b41b5d2_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3239_2022-4-21-20-01-30_pcap.csv
------------------------------
042122081929_new.csv diag_log_20220421_203616_b0e4a15ac89caff782b5f2af1b41b5d2_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3239_2022-4-21-20-12-19_pcap.csv
no ret
042122084330_new.csv diag_log_20220421_205559_b0e4a15ac89caff782b5f2af1b41b5d2_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3239_2022-4-21-20-18-34_pcap.csv
------------------------------
042122090027_new.csv diag_log_20220421_211228_b0e4a15ac89caff782b5f2af1b41b5d2_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3239_2022-4-21-20-41-02_pcap.csv
------------------------------
042122091753_new.csv diag_log_20220421_212534_b0e4a15ac89caff782b5f2af1b41b5d2_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3239_2022-4-21-21-16-53_pcap.csv
------------------------------
042122093019_new.csv diag_log_20220421_214426_b0e4a15ac89caff782b5f2af1b41b5d2_Xiaomi-M2007J3S

In [77]:
analysis_target_dir('/home/wmnlab/D/xm05/0421' , 'xm05result.csv')

042122080032_new.csv diag_log_20220421_201714_e90825d9912094c6845605a5069affae_Xiaomi-M2007J3SY_46692.mi2log.txt_3.csv DL_3241_2022-4-21-20-01-30_pcap.csv
------------------------------
042122081941_new.csv diag_log_20220421_203609_e90825d9912094c6845605a5069affae_Xiaomi-M2007J3SY_46692.mi2log.txt_3.csv DL_3241_2022-4-21-20-18-34_pcap.csv
------------------------------
042122084310_new.csv diag_log_20220421_205602_e90825d9912094c6845605a5069affae_Xiaomi-M2007J3SY_46692.mi2log.txt_3.csv DL_3241_2022-4-21-20-41-02_pcap.csv
------------------------------
042122090013_new.csv diag_log_20220421_211230_e90825d9912094c6845605a5069affae_Xiaomi-M2007J3SY_46692.mi2log.txt_3.csv DL_3241_2022-4-21-20-57-10_pcap.csv
------------------------------
042122091755_new.csv diag_log_20220421_212555_e90825d9912094c6845605a5069affae_Xiaomi-M2007J3SY_46692.mi2log.txt_3.csv DL_3241_2022-4-21-21-16-53_pcap.csv
------------------------------
042122093026_new.csv diag_log_20220421_214450_e90825d9912094c6845605a5

In [78]:
analysis_target_dir('/home/wmnlab/D/xm08/0421' , 'xm08result.csv')

042122080025_new.csv diag_log_20220421_201710_cb85f04bd99888b915d5a493b80eb313_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3245_2022-4-21-20-10-30_pcap.csv
------------------------------
042122081939_new.csv diag_log_20220421_203636_cb85f04bd99888b915d5a493b80eb313_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3245_2022-4-21-20-12-18_pcap.csv
no ret
042122084321_new.csv diag_log_20220421_205604_cb85f04bd99888b915d5a493b80eb313_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3245_2022-4-21-20-18-34_pcap.csv
------------------------------
042122090005_new.csv diag_log_20220421_211212_cb85f04bd99888b915d5a493b80eb313_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3245_2022-4-21-20-57-10_pcap.csv
------------------------------
042122091749_new.csv diag_log_20220421_212734_cb85f04bd99888b915d5a493b80eb313_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3245_2022-4-21-21-23-23_pcap.csv
no ret
042122093018_new.csv diag_log_20220421_214438_cb85f04bd99888b915d5a493b80eb313_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv

In [79]:
analysis_target_dir('/home/wmnlab/D/xm11/0421' , 'xm11result.csv')

042122080043_new.csv diag_log_20220421_201659_2d6caace15d954911dc1a16128c9bc7f_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3251_2022-4-21-20-01-30_pcap.csv
------------------------------
042122081938_new.csv diag_log_20220421_203619_2d6caace15d954911dc1a16128c9bc7f_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3251_2022-4-21-20-18-34_pcap.csv
------------------------------
042122084318_new.csv diag_log_20220421_205558_2d6caace15d954911dc1a16128c9bc7f_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3251_2022-4-21-20-41-02_pcap.csv
------------------------------
042122090034_new.csv diag_log_20220421_211222_2d6caace15d954911dc1a16128c9bc7f_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3251_2022-4-21-20-57-10_pcap.csv
------------------------------
042122091752_new.csv diag_log_20220421_212524_2d6caace15d954911dc1a16128c9bc7f_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3251_2022-4-21-21-16-53_pcap.csv
------------------------------
042122093005_new.csv diag_log_20220421_214500_2d6caace15d954911dc1a161

In [80]:
analysis_target_dir('/home/wmnlab/D/xm13/0421' , 'xm13result.csv')

042122080047_new.csv diag_log_20220421_201723_f0f24edebf24444354f19f7306a5f253_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3255_2022-4-21-20-04-45_pcap.csv
------------------------------
042122081928_new.csv diag_log_20220421_203621_f0f24edebf24444354f19f7306a5f253_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3255_2022-4-21-20-18-34_pcap.csv
------------------------------
042122084315_new.csv diag_log_20220421_205601_f0f24edebf24444354f19f7306a5f253_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3255_2022-4-21-20-41-02_pcap.csv
------------------------------
042122090044_new.csv diag_log_20220421_211219_f0f24edebf24444354f19f7306a5f253_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3255_2022-4-21-20-57-10_pcap.csv
------------------------------
042122091750_new.csv diag_log_20220421_212737_f0f24edebf24444354f19f7306a5f253_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3255_2022-4-21-21-16-53_pcap.csv
------------------------------


In [81]:
analysis_target_dir('/home/wmnlab/D/xm14/0421' , 'xm14result.csv')

042122080017_new.csv diag_log_20220421_201716_b1d46285fe5943f4381a649c8f15d3bd_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3257_2022-4-21-20-1-30_pcap.csv
------------------------------
042122081940_new.csv diag_log_20220421_203636_b1d46285fe5943f4381a649c8f15d3bd_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3257_2022-4-21-20-18-34_pcap.csv
------------------------------
042122084322_new.csv diag_log_20220421_205604_b1d46285fe5943f4381a649c8f15d3bd_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3257_2022-4-21-20-41-2_pcap.csv
------------------------------
042122085955_new.csv diag_log_20220421_211216_b1d46285fe5943f4381a649c8f15d3bd_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3257_2022-4-21-20-57-10_pcap.csv
------------------------------
042122091747_new.csv diag_log_20220421_212524_b1d46285fe5943f4381a649c8f15d3bd_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3257_2022-4-21-21-16-53_pcap.csv
------------------------------
042122093028_new.csv diag_log_20220421_214440_b1d46285fe5943f4381a649c8f

In [82]:
analysis_target_dir('/home/wmnlab/D/xm15/0421' , 'xm15result.csv')

042122080029_new.csv diag_log_20220421_201707_6b318ce3f2c55caa48bb83db829315bf_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3259_2022-4-21-20-01-30_pcap.csv
------------------------------
042122081940_new.csv diag_log_20220421_203540_6b318ce3f2c55caa48bb83db829315bf_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3259_2022-4-21-20-18-34_pcap.csv
------------------------------
042122084313_new.csv diag_log_20220421_205601_6b318ce3f2c55caa48bb83db829315bf_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3259_2022-4-21-20-41-02_pcap.csv
------------------------------
042122090020_new.csv diag_log_20220421_211229_6b318ce3f2c55caa48bb83db829315bf_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3259_2022-4-21-20-57-10_pcap.csv
------------------------------
042122091754_new.csv diag_log_20220421_212518_6b318ce3f2c55caa48bb83db829315bf_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3259_2022-4-21-21-16-53_pcap.csv
------------------------------
042122093020_new.csv diag_log_20220421_214446_6b318ce3f2c55caa48bb83db

In [83]:
analysis_target_dir('/home/wmnlab/D/xm16/0421' , 'xm16result.csv')

042122080040_new.csv diag_log_20220421_201718_34a716204ea399365b40d88700bc417d_Xiaomi-M2007J3SY_46692.mi2log.txt_3.csv DL_3261_2022-4-21-20-01-30_pcap.csv
------------------------------
042122081943_new.csv diag_log_20220421_203544_34a716204ea399365b40d88700bc417d_Xiaomi-M2007J3SY_46692.mi2log.txt_3.csv DL_3261_2022-4-21-20-18-34_pcap.csv
------------------------------
042122084322_new.csv diag_log_20220421_205555_34a716204ea399365b40d88700bc417d_Xiaomi-M2007J3SY_46692.mi2log.txt_3.csv DL_3261_2022-4-21-20-41-02_pcap.csv
------------------------------
042122085940_new.csv diag_log_20220421_211210_34a716204ea399365b40d88700bc417d_Xiaomi-M2007J3SY_46692.mi2log.txt_3.csv DL_3261_2022-4-21-20-57-10_pcap.csv
------------------------------
042122091747_new.csv diag_log_20220421_212553_34a716204ea399365b40d88700bc417d_Xiaomi-M2007J3SY_46692.mi2log.txt_3.csv DL_3261_2022-4-21-21-16-53_pcap.csv
------------------------------
042122093027_new.csv diag_log_20220421_214453_34a716204ea399365b40d887

In [84]:
analysis_target_dir('/home/wmnlab/D/xm17/0421' , 'xm17result.csv')

042122080022_new.csv diag_log_20220421_201704_b49e2a4249eb8bec2bf543f7d8be9422_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3263_2022-4-21-20-01-30_pcap.csv
------------------------------
042122081937_new.csv diag_log_20220421_203557_b49e2a4249eb8bec2bf543f7d8be9422_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3263_2022-4-21-20-18-34_pcap.csv
------------------------------
042122084317_new.csv diag_log_20220421_205601_b49e2a4249eb8bec2bf543f7d8be9422_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3263_2022-4-21-20-41-02_pcap.csv
------------------------------
042122090052_new.csv diag_log_20220421_211212_b49e2a4249eb8bec2bf543f7d8be9422_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3263_2022-4-21-20-57-10_pcap.csv
------------------------------
042122091921_new.csv diag_log_20220421_212521_b49e2a4249eb8bec2bf543f7d8be9422_Xiaomi-M2007J3SY_46697.mi2log.txt_3.csv DL_3263_2022-4-21-21-16-53_pcap.csv
------------------------------
042122093013_new.csv diag_log_20220421_214458_b49e2a4249eb8bec2bf543f7

In [156]:
def analysis_rtt_ul(dirpath , outputcsvfilename):
    outputfilecsv = open(outputcsvfilename, mode='w')
    cellinfodir = sorted(os.listdir(os.path.join(dirpath, "cellinfo")))
    mi2logdir = sorted(os.listdir(os.path.join(dirpath, "mi2log")))
    pcapdir = sorted(os.listdir(os.path.join(dirpath, "pcap")))
    rtt_list_list = []
    # pcapdir = sorted(os.listdir(os.path.join(dirpath, "server_dl_pcap")))
    assert(len(cellinfodir) == len(mi2logdir) == len(pcapdir) )
    for cellinfo, mi2log, pcapcsv in zip(cellinfodir, mi2logdir, pcapdir):
        pcapdf = read_pcapcsv(os.path.join(os.path.join(dirpath, "pcap"), pcapcsv))
        rtt_l = get_ul_rtt(pcapdf=pcapdf)
        rtt_list_list.append(rtt_l)
    return rtt_list_list

def analysis_rtt_dl(dirpath , outputcsvfilename):
    outputfilecsv = open(outputcsvfilename, mode='w')
    cellinfodir = sorted(os.listdir(os.path.join(dirpath, "cellinfo")))
    mi2logdir = sorted(os.listdir(os.path.join(dirpath, "mi2log")))
    pcapdir = sorted(os.listdir(os.path.join(dirpath, "server_dl_pcap")))
    rtt_list_list = []
    # pcapdir = sorted(os.listdir(os.path.join(dirpath, "server_dl_pcap")))
    assert(len(cellinfodir) == len(mi2logdir) == len(pcapdir) )
    for cellinfo, mi2log, pcapcsv in zip(cellinfodir, mi2logdir, pcapdir):
        pcapdf = read_pcapcsv(os.path.join(os.path.join(dirpath, "server_dl_pcap"), pcapcsv))
        rtt_l = get_dl_rtt(pcapdf=pcapdf)
        if len(rtt_l):
            rtt_list_list.append(rtt_l)
        else:
            print("pass", pcapcsv)
    return rtt_list_list

In [109]:
rtt_list_list = analysis_rtt_ul('/home/wmnlab/D/xm17/0421' , 'xm17result.csv')

100%|██████████| 113282/113282 [00:01<00:00, 66832.14it/s]


In [110]:
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

0.0022958098529886
0.0020992860314391136
0.002239537826268137
0.0031430371604491175
0.002073635519515989
0.0026928110724385376


In [ ]:
0.0022958098529886,
0.0020992860314391136,
0.002239537826268137,
0.0031430371604491175,
0.002073635519515989,
0.0026928110724385376

In [111]:
rtt_list_list = analysis_rtt_ul('/home/wmnlab/D/xm16/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 167834/167834 [00:02<00:00, 66619.63it/s]

0.0012157592598182722
0.0015379145757260999
0.0012331780089240851
0.001267568938629481
0.0016397090033104622
0.0012983877106000103


In [112]:
rtt_list_list = analysis_rtt_ul('/home/wmnlab/D/xm15/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 117249/117249 [00:01<00:00, 73938.71it/s]


0.0021803911684084207
0.0027819987993997018
0.002495996605520324
0.0028025237047015115
0.002767760988568681
0.0019890191303168106


In [113]:
rtt_list_list = analysis_rtt_ul('/home/wmnlab/D/xm14/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 109604/109604 [00:01<00:00, 70144.63it/s]

0.002187876122888973
0.0021344162353877957
0.0021620299663729706
0.0023347975982685107
0.0022324831978095044
0.0027749020800851986


In [114]:
rtt_list_list = analysis_rtt_ul('/home/wmnlab/D/xm13/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 62125/62125 [00:00<00:00, 83316.80it/s]

0.0020961336477987418
0.002143924321799204
0.0022632029885503503
0.0026924216428826793
0.0018676250256130103


In [115]:
rtt_list_list = analysis_rtt_ul('/home/wmnlab/D/xm11/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 116966/116966 [00:01<00:00, 70602.67it/s]

0.00225479043820094
0.0016916300831616702
0.0021349464774006884
0.0031604027627207313
0.0027815483368421124
0.002914714770026565


In [117]:
rtt_list_list = analysis_rtt_ul('/home/wmnlab/D/xm08/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 126016/126016 [00:01<00:00, 72566.87it/s]

0.002033140126661713
0.002672841175097544
0.0016216115621424305
0.0026640271156431403
0.0022612042619912895
0.0028272384987113423


In [118]:
rtt_list_list = analysis_rtt_ul('/home/wmnlab/D/xm05/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 165445/165445 [00:02<00:00, 65287.24it/s]


0.0012364712769516262
0.0012533333333333372
0.0012785837268817692
0.0014029782577903752
0.0013413835887433802
0.001208538885926898


In [119]:
rtt_list_list = analysis_rtt_ul('/home/wmnlab/D/xm04/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 104194/104194 [00:01<00:00, 78837.99it/s]

0.0021837319305753776
0.0029277394528959373
0.0022871454980261142
0.0023739150182902034
0.0020394662813102184
0.004199574023400627


In [157]:
rtt_list_list = analysis_rtt_dl('/home/wmnlab/D/xm04/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 50307/50307 [00:00<00:00, 60212.67it/s]

0.04130360746823299
0.052163499999999995
0.02819942189715392
0.027157067408027775
0.02780433980091102
0.08050255344990546


In [158]:
rtt_list_list

[array([[Timestamp('2022-04-21 20:04:35.045967'), 0.000116],
        [Timestamp('2022-04-21 20:04:35.062133'), 0.016166],
        [Timestamp('2022-04-21 20:04:35.081949'), 0.016569],
        ...,
        [Timestamp('2022-04-21 20:12:18.317179'), 0.872252],
        [Timestamp('2022-04-21 20:12:18.387126'), 0.898986],
        [Timestamp('2022-04-21 20:12:18.402177'), 0.832063]], dtype=object),
 array([[Timestamp('2022-04-21 20:12:22.377872'), 0.019857],
        [Timestamp('2022-04-21 20:12:22.381037'), 0.022041],
        [Timestamp('2022-04-21 20:12:22.381898'), 0.022738],
        [Timestamp('2022-04-21 20:12:22.410187'), 0.050286],
        [Timestamp('2022-04-21 20:12:22.412036'), 0.051727],
        [Timestamp('2022-04-21 20:12:22.412860'), 0.052049],
        [Timestamp('2022-04-21 20:12:22.416943'), 0.055997],
        [Timestamp('2022-04-21 20:12:22.417557'), 0.05532],
        [Timestamp('2022-04-21 20:12:22.422791'), 0.05976],
        [Timestamp('2022-04-21 20:12:22.422908'), 0.058962

In [159]:
rtt_list_list = analysis_rtt_dl('/home/wmnlab/D/xm05/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 89399/89399 [00:01<00:00, 64826.15it/s]

0.018802870441517246
0.01788516916309755
0.01890449898161619
0.01797449461933253
0.018035767021989037
0.018331589684068665


In [160]:
rtt_list_list = analysis_rtt_dl('/home/wmnlab/D/xm08/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 1/1 [00:00<00:00, 2146.52it/s]


pass DL_3245_2022-4-21-20-12-18_pcap.csv


100%|██████████| 1/1 [00:00<00:00, 2389.92it/s]


pass DL_3245_2022-4-21-21-23-23_pcap.csv


100%|██████████| 67821/67821 [00:01<00:00, 65938.08it/s]

0.02984019484766769
0.026820595851287838
0.028029158794333126
0.026159220647577135


In [161]:
rtt_list_list = analysis_rtt_dl('/home/wmnlab/D/xm09/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

AssertionError: 

In [162]:
rtt_list_list = analysis_rtt_dl('/home/wmnlab/D/xm11/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 64866/64866 [00:00<00:00, 70169.12it/s]

0.030838224776500604
0.030128250086012345
0.02815675741174602
0.029050454806715198
0.030070997552949032
0.02992807474867768


In [163]:
rtt_list_list = analysis_rtt_dl('/home/wmnlab/D/xm13/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 43564/43564 [00:00<00:00, 76049.67it/s]


0.033190700670435246
0.026930125096163483
0.026015097949974756
0.026957216266414657
0.033989277712952064


In [164]:
rtt_list_list = analysis_rtt_dl('/home/wmnlab/D/xm14/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 59178/59178 [00:00<00:00, 66942.71it/s]


0.02958448993923029
0.02775751451273484
0.028042349861200144
0.027680240342897074
0.030070098407682853
0.035854374156918244


In [165]:
rtt_list_list = analysis_rtt_dl('/home/wmnlab/D/xm15/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 64886/64886 [00:01<00:00, 53427.23it/s]

0.027911638586839548
0.02794213433833618
0.024507189192914648
0.028654384899195993
0.03001312217779417
0.028943155905939617


In [166]:
rtt_list_list = analysis_rtt_dl('/home/wmnlab/D/xm16/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 90198/90198 [00:01<00:00, 58218.29it/s]


0.019279587406869043
0.04216269352996505
0.018798835649725967
0.018115480122253835
0.01957717697385948
0.017373167068771494


In [167]:
rtt_list_list = analysis_rtt_dl('/home/wmnlab/D/xm17/0421' , 'xm17result.csv')
for i in range(len(rtt_list_list)):
    print(np.mean(rtt_list_list[i][:, 1]))

100%|██████████| 61388/61388 [00:00<00:00, 65556.74it/s]

0.030366692666266498
0.028790416523112042
0.026569471313508924
0.030344547962109322
0.029015460903964337
0.030878764722978815


[array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64)]